In [1]:
import pandas as pd
from copy import deepcopy
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
from tkinter import *
from tkinter.ttk import Combobox
from time import sleep
from PIL import Image, ImageTk

In [2]:
stocks_data = pd.read_csv("../datasets/stocks.csv")
stocks_data['Date'] = stocks_data['Date'].astype('datetime64[ns]')
stocks_data = stocks_data.set_index('Date')
stocks_data.head()

,Company,Open,High,Low,Close,Volume,OpenInt
Date,,,,,,,
1999-11-18,a,30.713,33.754,27.002,29.702,66277506,0
1999-11-19,a,28.986,29.027,26.872,27.257,16142920,0
1999-11-22,a,27.886,29.702,27.044,29.702,6970266,0
1999-11-23,a,28.688,29.446,27.002,27.002,6332082,0
1999-11-24,a,27.083,28.309,27.002,27.717,5132147,0


In [3]:
companies = deepcopy(stocks_data.Company)
companies.drop_duplicates(inplace=True)
companies.reset_index(drop=True,inplace=True)

In [4]:
comp_list = list(companies)

In [5]:
comp_stocks=pd.read_csv("filtered_companies.csv")
comp_stocks['Symbol']=comp_stocks.Symbol.str.lower()

In [2]:
word_data = pd.read_csv("../datasets/words_dates_list.csv")
word_data.drop_duplicates(inplace=True)
word_data.reset_index(drop=True,inplace=True)
keywords = deepcopy(word_data.keyword)
word_list = list(keywords)

In [11]:
# GUI for stock details
window= Tk()
window.configure(background='#ffffff')
window.geometry('1366x768')

l = Label(window, text="Stock Details of a Company",font='Sans-serif 30 bold',justify = CENTER)
l.configure(background='#ffffff')
l.grid(row=0, column=0,columnspan=30,pady=(10,10), padx=(10,10))

canvas = Canvas(window, bg="white", height=500, width=900)
canvas.grid(row=1,rowspan=20,padx=40, pady=30)

c = Canvas(window, bg="white", height=200, width=320)
c.grid(row=18,column=1,columnspan=5,padx=0, pady=10)

# graph plot
def plot(company):
    cp = stocks_data[stocks_data.Company==company].Close
    plt.figure()
    cp.plot(figsize = (10,5))
    plt.legend()
    plt.title("company : "+company,fontsize=20)
    PATH = "graph_images/"+company+".jpeg"
    plt.savefig(PATH)
    plt.cla()
    plt.close('all')
    return PATH
    
def select():
    comp = var.get()
    path = plot(comp)
    canvas.delete("all")
    c.delete('all')
    image = Image.open(path)
    img = ImageTk.PhotoImage(image)
    canvas.create_image(450,250, image=img)
    label = Label(image=img)
    label.image = img 
    image.close()
    
    x = comp_stocks[comp_stocks.Symbol==comp]
    x.reset_index(drop=True,inplace=True)
    if(x.empty):
        c.create_text(120,35,fill="Black",font="Times 13 italic bold",
                            text="Company details not found")
        
    else:
        c.create_text(30,15,fill="Black",font="Times 12 bold",
                            text="Name:")

        c.create_text(180,15,fill="Black",font="Times 10",
                            text=x.Name[0])
        
        c.create_text(40,55,fill="Black",font="Times 12 bold",
                            text="LastSale:")

        c.create_text(105,55,fill="Black",font="Times 10",
                            text=x.LastSale[0])
        
        c.create_text(47,95,fill="Black",font="Times 12 bold",
                            text="MarketCap:")

        c.create_text(125,95,fill="Black",font="Times 10",
                            text=x.MarketCap[0])
        
        c.create_text(37,135,fill="Black",font="Times 12 bold",
                            text="IPOyear:")

        c.create_text(105,135,fill="Black",font="Times 10",
                            text=x.IPOyear[0])
        
        c.create_text(29,175,fill="Black",font="Times 12 bold",
                            text="Sector:")

        c.create_text(108,175,fill="Black",font="Times 10",
                            text=x.Sector[0])
        
    
        

def insert_into_entry(Event = None):
    items = list(map(int, lb.curselection()))
    if items != ():
        selected_item=[lb.get(items[i]) for i in range(len(items))]
        print(selected_item)


var = StringVar(window)
var.set(comp_list[0])
                     
lbl1 = Label(window, text="Companies",font='Helvetica 12 bold')
lbl1.configure(background='#ffffff')
lbl1.grid(row=10,column=1,padx=20)

combo = Combobox(window,textvariable=var)
combo['values']= comp_list
combo.grid(row=10,column=2,padx=20)

lbl2 = Label(window, text="Keywords",font='Helvetica 12 bold')
lbl2.configure(background='#ffffff')
lbl2.grid(row=11,column=1,padx=20)


sb = Scrollbar(window, orient=VERTICAL)
lb = Listbox(window,selectmode=MULTIPLE, yscrollcommand=sb.set)
lb.insert(END,*word_list)
sb.grid(row=12,column=2,sticky=(NS,W))
sb.config(command=lb.yview)
lb.grid(row=12,column=1,sticky=E)
lb.bind('<Return>',insert_into_entry)

button = Button(window, text="submit",fg='white', bg='#13ade0',relief=GROOVE,activebackground='#1da1f2',activeforeground='white',command=select, width = 8)
button.grid(row=14,column=1)

w = Label(window, text="Company Details", font=("Helvetica", 16))
w.configure(background='#ffffff')
w.grid(row=17,column=1)
window.mainloop()
